# MLB Hall of Fame Predictor

**INFO 523 Data Mining**  
**Authors: Austin Cortopassi, David Pelley, Nathan Harville**  
**Professor: Dr. Greg Chism**

**Questions:**
Can we predict whether a player will be inducted into the MLB Hall of Fame based on their career statistics and achievements?  
What features (e.g., batting stats, pitching metrics, postseason performance) are most predictive of a Hall of Fame induction?

In [1]:
import numpy as np
import pandas as pd

# Data Prep

In [28]:
# load all datasets into separate variables
people = pd.read_csv("..\data\People.csv")
batting = pd.read_csv("..\data\Batting.csv")
pitching = pd.read_csv("..\data\Pitching.csv")
fielding = pd.read_csv("..\data\Fielding.csv")
teams = pd.read_csv("..\data\Teams.csv")
batting_post = pd.read_csv("..\data\BattingPost.csv")
pitching_post = pd.read_csv("..\data\PitchingPost.csv")
fielding_post = pd.read_csv("..\data\FieldingPost.csv")
hall_of_fame = pd.read_csv("..\data\HallOfFame.csv")
awards_players = pd.read_csv("..\data\AwardsPlayers.csv")
awards_players_shared = pd.read_csv("..\data\AwardsSharePlayers.csv")

In [ ]:
# Prep individual datasets before merging

# drop unused columns from people dataset such as alternate IDs, death and birth information
people = people.drop(columns=['birthYear','birthMonth','birthDay','birthCountry','birthState','birthCity','deathYear','deathMonth','deathDay','deathCountry','deathState','deathCity','retroID','bbrefID'])

# drop unused columns from batting dataset; stint and league ID
batting = batting.drop(columns=['stint','lgID'])

# drop unused columns from pitching dataset; stint and league ID
pitching = pitching.drop(columns=['stint','lgID'])

# drop unused columns from fielding dataset; stint and league ID
fielding = fielding.drop(columns=['stint','lgID'])

# drop unused columns from teams dataset; only want to include the yearID, teamID, Wins, Losses, and their postseason performance
teams = teams[['yearID','teamID','Rank','W','L','DivWin','WCWin','LgWin','WSWin']]

# drop unused columns from postseason batting dataset; leagueID and round
batting_post = batting_post.drop(columns=['round','lgID'])

# drop unused columns from postseason pitching dataset; leagueID and round
pitching_post = pitching_post.drop(columns=['round','lgID'])

# drop unused columns from postseason fielding dataset; leagueID and round
fielding_post = fielding_post.drop(columns=['round','lgID'])

# drop unused columns from hall of fame dataset; voted by, category, and needed note
hall_of_fame = hall_of_fame.drop(columns=['votedBy','category','needed_note'])

# drop unused columns in awards and awards shared
awards_players = awards_players.drop(columns=['lgID','notes'])
awards_players_shared = awards_players_shared.drop(columns=['lgID'])

# # Merge batting with people on playerID (inner join keeps only players with batting data)
# df = pd.merge(batting, people, on="playerID", how="inner")

# # Merge pitching
# df = pd.merge(df, pitching, on="playerID", how="left")  # left join to keep batting + people

# # Merge fielding
# df = pd.merge(df, fielding, on="playerID", how="left")

# # Merge hall of fame
# df = pd.merge(df, hall_of_fame, on="playerID", how="left")

# # Now df has info combined from all datasets on playerID
# print(df.shape)
# print(df.head())